In [1]:
import transformers
import torch, math, einops
from torch import nn


from src.encoder import GPT2Encoder
from src.decoder import GPT2Decoder
from src.GPT2 import GPT2_Block, GPT2
from src import BlockGenerator
from src import Tokenizer, random_unidirectional_graph_maker, linear_unidirectional_graph_maker
from src.utils import validation

In [2]:
from datasets import load_dataset

train = load_dataset("wikipedia", "20220301.simple",split="train[:80%]")
valid = load_dataset("wikipedia", "20220301.simple",split="train[80%:]")

Found cached dataset wikipedia (/Users/francescosacco/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)
Found cached dataset wikipedia (/Users/francescosacco/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


In [3]:
device='cpu'
#device = 'mps'  if torch.backends.mps.is_available() else 'cpu'
device = 'cuda' if torch.cuda.is_available() else device

dK = 64
dV = 64
heads = 12
d_Embedding = dK*heads

tokenizer = Tokenizer('gpt2',device=device)
encoder = GPT2Encoder()
decoder = GPT2Decoder(encoder)
block_generator = BlockGenerator(GPT2_Block)
model = GPT2(tokenizer, encoder, block_generator, decoder)

pretrained=True
if pretrained:
    pretrained = transformers.GPT2LMHeadModel.from_pretrained('gpt2').to(device)
    model.load_from_original(pretrained)

graph_maker = linear_unidirectional_graph_maker(500)
#graph_maker = random_unidirectional_graph_maker(50,50)

In [4]:
from src.decoder import Loss

loss_function=Loss(decoder)
ce_loss=nn.CrossEntropyLoss()
lr=1e-9
gamma=0.99

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma)

In [5]:
graph_maker = linear_unidirectional_graph_maker(500)

validation(valid,model,loss_function,graph_maker,10)


/Users/francescosacco/Documents/Language_CA/src/attention.py:122: UserWarning: scatter_reduce() is in beta and the API may change at any time. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525473998/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1615.)
  translation = translation.scatter_reduce(0, receivers.repeat(heads, 1).t(), att, reduce='amax', include_self=False)


tensor([3.8487])

In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('gpt2')
x=tokenizer.encode("the capital of france is paris",add_special_tokens=False,return_tensors='pt').to(device)
out=pretrained(x).logits[0]
print(out.shape)
asd=tokenizer.decode(out.argmax(-1))
#ce_loss(out[:-1],x[1:])

generated=pretrained.generate(input_ids=x)
#print(asd)
print(asd,generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


torch.Size([8, 50257])


/Users/francescosacco/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


, of theaternal, thel, tensor([[ 1169,  3139,   286,  1216,   590,   318,  1582,   271,    11,   290,
           262,  1748,   318,  1444,   262,  1748,   286,   262, 11858,    11]])


In [ ]:
asd=transformers.GPT2Model.from_pretrained('gpt2')

In [60]:
pretrained??

Signature:      pretrained(*input, **kwargs)
Type:           GPT2LMHeadModel
String form:   
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Co

In [5]:
n_epochs = 2
model.train()
losses = []
for _ in range(n_epochs):
    for page in train:
        text = page['text']
        nodes = tokenizer(text).to(device)
        nodes, target = nodes[:-3], nodes[1:-2]

        if isinstance(encoder, GPT2Encoder):
            nodes = nodes[:1023]
            target = target[:1023]
        edge_index = graph_maker(len(nodes))

        optimizer.zero_grad()  # reinitialize the gradient to zero
        
        prediction=model(nodes,edge_index)
        loss=loss_function(prediction,target)
        #prediction=pretrained(nodes)[0]
        #loss=ce_loss(prediction,target)
        
        print(loss.item())
        losses.append(loss.item())
        loss.backward()

        optimizer.step()


Token indices sequence length is longer than the specified maximum sequence length for this model (3814 > 1024). Running this sequence through the model will result in indexing errors


2.54327654838562
2.1318130493164062
3.1408321857452393
3.305172920227051
3.1729109287261963
3.1030735969543457
3.188854694366455


KeyboardInterrupt: 

In [8]:
from src.graph_initialization import linear_unidirectional_graph_maker
text = "Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry. "

print(text, end='')
model.generate(text, 20, graph_maker,temperature=1)


Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.   After rescuing Tok'ra senses, Chronus climbed to Promethean aid.   A threat

"Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry. \xa0 After rescuing Tok'ra senses, Chronus climbed to Promethean aid. \xa0 A threat"